## Packages

In [20]:
from __future__ import print_function
import os
import sys
import pickle

import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


## Constants

In [21]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SAMPLE_SPREADSHEET_ID = '1EHdPEQm9Suj9UZJd4n8vACkXLdzx4cQEun4Png__hTI'
SHEETS_NAMES = ['Buts', 'Dividendes', 'Dividendes Dernière journée', 'PPF', 'Passe Dé', 'Penalty Reussi',
                'CARTONS JAUNE', 'ARRETS GARDIEN', 'TITULARISATION', 'MATCH JOUÉ', 'PRIX', 'FBREF']

## Functions

In [22]:
def get_authenticated_service():
    creds = None
    if os.path.exists('../analysis/token.json'):
        creds = Credentials.from_authorized_user_file('../analysis/token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('../analysis/credentials.json', SCOPES)
            creds = flow.run_local_server(port=8080)
        with open('../analysis/token.json', 'w') as token:
            token.write(creds.to_json())

    return build('sheets', 'v4', credentials=creds)


def get_all_sheet_data(service):
    all_data = {}
    sheet = service.spreadsheets()

    for sheet_name in SHEETS_NAMES:
        range_name = f'{sheet_name}!A1:AJ500'  # Construct the range name
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=range_name).execute()
        values = result.get('values', [])

        if not values:
            continue

        all_data[sheet_name] = values

    return all_data


## SmokeTest

In [23]:
service = get_authenticated_service()
df = get_all_sheet_data(service)

## PpfData


In [24]:
ppf_df = pd.DataFrame(df.get('PPF'))

#set column names equal to values in row index position 0
ppf_df.columns = ppf_df.iloc[0]
#remove first row from DataFrame
ppf_df = ppf_df[1:]

ppf_df

,Numéro,NOM,PRIX,PPF,POSTE
1,48,Abakar SYLLA,4004600,63,Défenseur
2,185,Abdoul Kader BAMBA,2638333,9,Milieu
3,69,Abdoulaye TOURé,3134778,49,Milieu
4,146,Ablie JALLOW,2402000,15,Milieu
5,265,Abou Lô,2499666,4,Défenseur
...,...,...,...,...,...
388,285,Youssouf FOFANA,7732300,3,Milieu
389,138,Youssouf NDAYISHIMIYE,5252000,18,Défenseur
390,125,Yunis ABDELHAMID,5480000,22,Défenseur
391,40,Yvon MVOGO,4571800,70,Gardien


## FbrefData

In [25]:
fbref_df = pd.DataFrame(df.get('FBREF'))

#set column names equal to values in row index position 0
fbref_df.columns = fbref_df.iloc[0]
#remove first row from DataFrame
fbref_df = fbref_df[1:]

fbref_df

,Clt,NOM,Nation,Pos,Équipe,Âge,Naissance,MJ,Titulaire,Min,...,Buts/90,PD/90,B+PD/90,B-PénM/90,B+PD-PénM/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90
1,1,Yunis Abdelhamid,ma MAR,DF,Reims,35-330,1987,2,2,180,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2,Salis Abdul Samed,gh GHA,MT,Lens,23-151,2000,2,2,180,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02
3,3,Laurent Abergel,fr FRA,MT,Lorient,30-204,1993,2,2,180,...,0.00,0.00,0.00,0.00,0.00,0.05,0.04,0.08,0.05,0.08
4,4,Zakaria Aboukhlal,ma MAR,"MT,AT",Toulouse,23-187,2000,2,2,171,...,01.05,0.00,01.05,0.53,0.53,0.86,0.05,0.92,0.45,0.50
5,5,Akor Adams,ng NGA,AT,Montpellier,23-207,2000,2,2,166,...,1.63,0.00,1.63,1.63,1.63,0.87,0.17,01.04,0.87,01.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,311,Warren Zaire-Emery,fr FRA,MT,Paris S-G,17-169,2006,2,2,180,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02
312,312,Denis Zakaria,ch SUI,DF,Monaco,26-277,1996,1,1,87,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
313,313,Mehdi Zeffane,dz ALG,DF,Clermont Foot,31-097,1992,2,2,151,...,0.00,0.00,0.00,0.00,0.00,0.21,0.07,0.28,0.21,0.28
314,314,Nathan Zeze,fr FRA,DF,Nantes,18-067,2005,1,1,90,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Save dataframe

In [26]:
# Obtenir le chemin du répertoire du notebook
current_path = os.getcwd()
directory_path = os.path.dirname(current_path) +'/data/external/'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

with open(directory_path + 'google_data', 'wb') as f:
    pickle.dump(df, f)